In [203]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

In [204]:
pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [205]:
import opendatasets as od

In [206]:
od.download("https://www.kaggle.com/datasets/chancev/carsforsale")


Skipping, found downloaded files in "./carsforsale" (use force=True to force download)


In [207]:
import pandas as pd


In [233]:
df = pd.read_csv('./carsforsale/cars_raw.csv')

del df["VIN"]
del df["Stock#"]
del df["InteriorColor"]
del df["StreetName"]
del df["SellerName"]
del df["Engine"]




df.head(3)

,Year,Make,Model,Used/New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerRating,SellerReviews,...,ValueForMoneyRating,ExteriorStylingRating,ReliabilityRating,ExteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Mileage
0,2019,Toyota,Sienna SE,Used,"$39,998",4.6,45,Dealer,3.3,3,...,4.4,4.6,4.7,Red,Front-wheel Drive,19,27,Gasoline,8-Speed Automatic,29403
1,2018,Ford,F-150 Lariat,Used,"$49,985",4.8,817,Dealer,4.8,131,...,4.6,4.8,4.7,Shadow Black,Four-wheel Drive,19,24,Gasoline,10-Speed Automatic,32929
2,2017,RAM,1500 Laramie,Used,"$41,860",4.7,495,Dealer,4.6,249,...,4.6,4.8,4.7,Granite Crystal Clearcoat Metallic,Four-wheel Drive,15,21,Gasoline,8-Speed Automatic,23173


In [234]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()
df[["Make"]] = enc.fit_transform(df[["Make"]])
df[["SellerType"]] = enc.fit_transform(df[["SellerType"]])
df[["State"]] = enc.fit_transform(df[["State"]])
df[["DealType"]] = enc.fit_transform(df[["DealType"]])
df[["ExteriorColor"]] = enc.fit_transform(df[["ExteriorColor"]])
df[["Drivetrain"]] = enc.fit_transform(df[["Drivetrain"]])
df[["FuelType"]] = enc.fit_transform(df[["FuelType"]])
df[["Transmission"]] = enc.fit_transform(df[["Transmission"]])
df[["Model"]] = enc.fit_transform(df[["Model"]])

df['Price'] = df['Price'].str.replace(',',''); 
df['Price'] = df['Price'].str.replace('$',''); 
df["Used/New"][0]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app


'Used'

In [235]:
for i in range(df["Used/New"].shape[0]):
  if df["Used/New"][i] != "New":
    df["Used/New"][i] = 0
  else:
    df["Used/New"][i] = 1


df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(999, inplace=True)

df.drop(df[df.Zipcode.str.isnumeric()== False].index, inplace = True)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [236]:
df.drop(df[df.Zipcode.str.isnumeric()== False].index, inplace = True)

df_train = df.iloc[:(int)(df.shape[0]*0.8), :]
df_test = df.iloc[(int)(df.shape[0]*0.8):, :]

df_train.head(10)

,Year,Make,Model,Used/New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerRating,SellerReviews,...,ValueForMoneyRating,ExteriorStylingRating,ReliabilityRating,ExteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Mileage
0,2019,38.0,1050.0,0,39998,4.6,45,0.0,3.3,3,...,4.4,4.6,4.7,694.0,6.0,19,27,7.0,47.0,29403
1,2018,12.0,427.0,0,49985,4.8,817,0.0,4.8,131,...,4.6,4.8,4.7,760.0,4.0,19,24,7.0,7.0,32929
2,2017,33.0,4.0,0,41860,4.7,495,0.0,4.6,249,...,4.6,4.8,4.7,370.0,4.0,15,21,7.0,47.0,23173
3,2021,15.0,151.0,0,28500,5.0,36,0.0,4.6,284,...,5.0,5.0,5.0,382.0,6.0,29,35,7.0,68.0,10598
4,2020,23.0,897.0,0,49000,4.8,76,0.0,4.8,4755,...,4.7,4.8,4.9,315.0,6.0,20,27,7.0,47.0,28137
5,2012,38.0,50.0,0,23541,4.7,34,0.0,4.4,1071,...,4.6,4.9,4.9,73.0,8.0,17,23,7.0,14.0,105469
6,2017,15.0,548.0,0,20995,4.6,200,0.0,4.4,2695,...,4.5,4.7,4.8,546.0,6.0,28,34,7.0,68.0,10458
7,2014,28.0,345.0,0,18985,4.8,176,0.0,4.4,2695,...,4.6,4.9,4.8,479.0,8.0,21,30,7.0,35.0,58157
8,2021,15.0,807.0,0,44299,4.8,63,0.0,4.9,237,...,4.7,4.7,4.8,652.0,2.0,19,26,7.0,54.0,14445
9,2020,9.0,280.0,0,46773,4.8,56,0.0,4.3,1366,...,4.7,4.9,4.9,872.0,8.0,15,24,7.0,47.0,25642


In [242]:
y_train = df_train['Price']
del df_train['Price']


y_test = df_test['Price']
del df_test['Price']

y_train.head(10)


0    39998
1    49985
2    41860
3    28500
4    49000
5    23541
6    20995
7    18985
8    44299
9    46773
Name: Price, dtype: object

In [245]:
regr_1 = DecisionTreeRegressor(max_depth=24)
regr_1.fit(df_train,y_train)

DecisionTreeRegressor(max_depth=24)

In [247]:
from sklearn.metrics import mean_absolute_percentage_error
a = regr_1.predict(df_test)
mean_absolute_percentage_error(y_test, a)




0.12777822655478888